In [2]:
!pip install langchain-community chromadb


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install langchain pypdf


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install sentence-transformers


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [4]:
# import os
# from getpass import getpass

# HF_TOKEN = getpass("Enter HF Token:")
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_TOKEN

In [5]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub

## Load and Chunk the data

Using the Langchain loader, you can load the document. For the chunking we will use Recursive character text splitter

In [23]:
# documents = PyPDFLoader("Tarun_CV.pdf").load()

path = "./data/"
loader = PyPDFDirectoryLoader(path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

## Index your document

In [24]:
# embeddings = HuggingFaceInferenceAPIEmbeddings(
#     api_key = HF_TOKEN,model_name = "thenlper/gte-large"
# )

emb_model = "BAAI/bge-small-en-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=emb_model)

In [25]:
db = Chroma.from_documents(texts, embeddings)

## Retriever - Normal Vector search retriever

In [26]:
retriever = db.as_retriever(
    search_kwargs={"k": 4}
)

Now the above retriever will retrieve 4 documents based on your query. But which among them is the most relevant, in order to rank that we use reranking.

## Time to Re-rank the documents

In [27]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder


### In our case we are using Cross-Encoders, that is open source and we shall use HuggingFace

A cross-encoder is a type of neural network architecture used in natural language processing tasks, particularly in the context of sentence or text pair classification. Its purpose is to evaluate and provide a single score or representation for a pair of input sentences, indicating the relationship or similarity between them. This is in contrast to other architectures like siamese networks or traditional encoder-decoder models.

https://medium.com/@sujathamudadla1213/what-is-cross-encoder-fec22b58f16c


In [28]:
cross_encoder_model="BAAI/bge-reranker-base"
cross_encoder_model="cross-encoder/ms-marco-MiniLM-L-2-v2"

model = HuggingFaceCrossEncoder(model_name=cross_encoder_model)

# from sentence_transformers import CrossEncoder
# model = CrossEncoder('model_name', max_length=512)

compressor = CrossEncoderReranker(model=model, top_n=3)
re_rank_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [29]:
re_rank_retriever.invoke("Who is Jonathan Loscalzo")

[Document(metadata={'page': 0, 'source': 'data/Jonathan Loscalzo - Resume.pdf'}, page_content='Jonath an\nLoscalzo\nSoftware\nEngineer\nLa\nPlata,\nBuenos\nAires,\nArgentina\njonathan.r.loscalzo@gmail.com\nJonathan\nLoscalzo\n|\nLinkedIn\nhttps://github.com/JonathanLoscalzo\n@jonathanloscalzo\n|\nMedium\nPROFILE\nProactive\nsoftware\ndeveloper\nwith\n10\nyears\nof\nexperience,\npassionate\nabout\ntechnological\ninnovation,\nespecially\nin\nArtiﬁcial\nIntelligence.\nDriven\nby\ncontinuous\nlearning,\nadaptability\nto\nnew\ntechnologies,\nand\nthriving\nin\nfast-paced\nenvironments.\nSeeking\na\nposition\nas\na\nSoftware\nor\nMachine\nLearning\nEngineer,'),
 Document(metadata={'page': 1, 'source': 'data/lekl101.pdf'}, page_content='2/KALEIDOSCOPE\nI S I SI S I SI S ell my Dreamsell my Dreamsell my Dreamsell my Dreamsell my Dreams\nGabriel Garcia Marquez was brought up by his\ngrandparents in Northern Columbia because his\nparents were poor and struggling. A novelist, short-\nstory writer

## Define LLM and its relevant prompt template

Here we will use HuggingFace LLM that is open source and free to inference

In [30]:
# model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-alpha",
#                        model_kwargs = {"temperature":0.5,
#                                        "max_new_tokens":512,
#                                        "max_length":64,
#                                        "return_full_text":False})

from langchain_community.llms import Ollama
model_kwargs={
    # "max_new_tokens": 512,
    "top_k": 2,
    "temperature": 0.5,
    # "repetition_penalty": 1.1,
    # "return_full_text":False,
}

model = Ollama(
    model="llama3:8b",
    **model_kwargs
)

In [31]:
query = "What is Tarun's contribution at AI Planet?"

In [32]:
template = """
<|system|>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [33]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

In [34]:
prompt

ChatPromptTemplate(input_variables=['context', 'query'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'query'], template="\n<|system|>\nYou are an AI Assistant that follows instructions extremely well.\nPlease be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT\n\nCONTEXT: {context}\n</s>\n<|user|>\n{query}\n</s>\n<|assistant|>\n"))])

## LCEL - Langchain Expression Language

In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [36]:
output_parser = StrOutputParser()

In [37]:
chain = (
    {"context": re_rank_retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [38]:
response = chain.invoke(query)

In [39]:
print(response)
# Tarun's contribution at AI Planet is not explicitly mentioned in the given context. However, it is mentioned that AI Planet currently has 16K+ followers on LinkedIn and 9K+ subscribers on YouTube, and Tarun is the lead curriculum contributor to the LLM Bootcamp, where he reached out to 11 speakers and led a group of 8 AI Ambassadors for the AI Changemaker program. It is possible that Tarun's contribution at AI Planet is related to these programs and initiatives, but it is not explicitly stated in the given context.

I don't know. The provided context does not mention a person named Tarun or any contributions made by them at AI Planet. The context appears to be related to Jonathan Loscalzo's resume, which mentions his skills and experiences in various technologies and projects. There is no information about Tarun or their involvement with AI Planet.


In [40]:
chain.invoke("Who is Jonathan Loscalzo?")

'Jonathan Loscalzo is a Software Engineer from La Plata, Buenos Aires, Argentina. He has 10 years of experience in the field and is passionate about technological innovation, especially in Artificial Intelligence.'